In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### assessment - object-oriented programming
1. simulate a server that's taking connections from the outside
1. simulate a load balancer that ensures that there are enough servers to serve those connections

**Define a Server class for simulation**
* Take care of the connections, represented by an id,(e.g. IP address of the device connecting to the server)
* For simulation, each connection creates a random amount of load in the server, between 1 and 10.

In [2]:
import random

class Server:
    def __init__(self):
        """Creates a new server instance, with no active connections."""
        self.connections = {}

    def add_connection(self, connection_id):
        """Adds a new connection to this server."""
        connection_load = random.random()*10+1
        # Add the connection to the dictionary with the calculated load
        self.connections[connection_id] = connection_load

    def close_connection(self, connection_id):
        """Closes a connection on this server."""
        # Remove the connection from the dictionary
        del self.connections[connection_id]

    def load(self):
        """Calculates the current load for all connections."""
        total = 0
        # Add up the load for each of the connections
        for load in self.connections.values():
            total += load
        return total

    def __str__(self):
        """Returns a string with the current load of the server"""
        return "{:.2f}%".format(self.load())
    

**Test some Server instances**   
* Create a Server instance   
* Add a connection to the server manually   
* Invoke methods to check the load values


In [3]:
server = Server()
server.add_connection("192.168.1.1")

print(server.load())

print(server)

8.174625199549567
8.17%


**Test closing a connection**   
* Close the connection added earlier using the Server class method   
* Check load values, should be 0

In [4]:
server.close_connection("192.168.1.1")
print(server.load())

0


**Define a LoadBalancing class for simulation**
* Start with one server
* When a connection gets added, it randomly selects an available server to serve that connection
* Pass the connection to the selected server
* Keep track of the ongoing connections to be able to close them

In [5]:
class LoadBalancing:
    def __init__(self):
        """Initialize the load balancing system with one server"""
        self.connections = {}
        self.servers = [Server()]

    def add_connection(self, connection_id):
        """Randomly selects a server and adds a connection to it."""
        self.ensure_availability()
        
        server = random.choice(self.servers)
        # Add the connection to the dictionary with the selected server
        self.connections[connection_id] = server
        # Add the connection to the server
        server.add_connection(connection_id)
        
    def close_connection(self, connection_id):
        """Closes the connection on the the server corresponding to connection_id."""
        # Find out the right server
        close_server = self.connections[connection_id]
        # Close the connection on the server
        close_server.close_connection(connection_id)
        # Remove the connection from the load balancer
        del self.connections[connection_id]

    def avg_load(self):
        """Calculates the average load of all servers"""
        # Sum the load of each server and divide by the amount of servers
        total_load = 0
        no_servers = len(self.servers)
        if no_servers > 0:
            for server in self.servers:
                for load in server.connections.values():
                    total_load += load
            return total_load / no_servers
        else:
            return 0

    def ensure_availability(self):
        """If the average load is higher than 50, spin up a new server"""
        if self.avg_load() > 50:
            print("Avg. server load {}, adding a server...".format(self.avg_load()))
            self.servers.append(Server())

    def __str__(self):
        """Returns a string with the load for each server."""
        loads = [str(server) for server in self.servers]
        return "[{}]".format(",".join(loads))

**Test with one incoming connection**
* Create an instance of LoadBalancing
* Add a connection to the LoadBalancing
* Use the class method to check the calculated output of loads from the server(s)

In [6]:
l = LoadBalancing()
l.add_connection("fdca:83d2::f20d")
print(l.avg_load())

4.934077787764557


**Add another server to the LoadBalancing**   
* Use the LoadBalancing class method and add another server    
* Check the output of average loads

In [7]:
l.servers.append(Server())
print(l.avg_load())

2.4670388938822785


**Close the connection**   
* Close the connection from the server assigned to
* Remove the connection information from the LoadBalancing   
* Check the output of average load value

In [8]:
l.close_connection("fdca:83d2::f20d")
print(l.avg_load())

0.0


**Automate the server addition**   
* Add a server to LoadBalancing automatically when the average load is more than 50%   
* Implement the automation in the class methods:   
    * ensure_availability()
    * add_connection(connection_id)

**Test the automation**   
* Add 50 connections
* Check each server's load for each addition
* Verify the automation condition(50) by checking the result of average load for LoadBalancing (entire server fleet) - should display <= 50

In [9]:
# testing by adding 50 connections 
for connection in range(50):
    l.add_connection(connection)
    print(l) # for each addition of connection

print("All connections added..")
print("Current avg. server load is {}".format(l.avg_load()))


[4.74%,0.00%]
[12.86%,0.00%]
[12.86%,5.60%]
[21.77%,5.60%]
[21.77%,7.69%]
[22.92%,7.69%]
[22.92%,15.09%]
[24.23%,15.09%]
[32.28%,15.09%]
[41.84%,15.09%]
[49.12%,15.09%]
[49.12%,23.29%]
[58.70%,23.29%]
[68.44%,23.29%]
[75.14%,23.29%]
[75.14%,28.15%]
Avg. server load 51.648004806181085, adding a server...
[82.85%,28.15%,0.00%]
[82.85%,34.70%,0.00%]
[93.23%,34.70%,0.00%]
[97.67%,34.70%,0.00%]
[97.67%,45.24%,0.00%]
[99.52%,45.24%,0.00%]
[99.52%,45.24%,3.35%]
[103.66%,45.24%,3.35%]
Avg. server load 50.74908884980726, adding a server...
[107.39%,45.24%,3.35%,0.00%]
[109.35%,45.24%,3.35%,0.00%]
[109.35%,45.24%,11.36%,0.00%]
[118.01%,45.24%,11.36%,0.00%]
[122.73%,45.24%,11.36%,0.00%]
[122.73%,52.45%,11.36%,0.00%]
[122.73%,52.45%,21.30%,0.00%]
[122.73%,52.45%,22.34%,0.00%]
[122.73%,61.68%,22.34%,0.00%]
Avg. server load 51.68641530017986, adding a server...
[122.73%,61.68%,29.21%,0.00%,0.00%]
[122.73%,61.68%,29.21%,6.13%,0.00%]
[122.73%,61.68%,29.21%,6.13%,9.01%]
[122.73%,61.68%,29.21%,6.13%,10.